In [2]:
import pandas as pd

df = pd.read_csv('./data/ehg_couples_families.csv')

df

,Average Monthly Household Income,EHG Amount (SGD)
0,Not more than 1500,120000
1,1501 to 2000,110000
2,2001 to 2500,105000
3,2501 to 3000,95000
4,3001 to 3500,90000
5,3501 to 4000,80000
6,4001 to 4500,70000
7,4501 to 5000,65000
8,5001 to 5500,55000
9,5501 to 6000,50000


In [27]:
import pandas as pd

# Load and clean EHG data
def load_ehg_data(filepath, is_couple=True):
    ehg_data = pd.read_csv(filepath)

    # Ensure the income bracket column is a string
    income_column = "Average Monthly Household Income" if is_couple else "Half of Average Monthly Household Income"
    ehg_data[income_column] = ehg_data[income_column].astype(str)

    # Create min and max income columns
    def parse_income_bracket(bracket):
        if "Not more than" in bracket:
            return 0, int(bracket.replace("Not more than ", "").replace(",", ""))
        elif "More than" in bracket:
            return int(bracket.replace("More than ", "").replace(",", "")), float("inf")
        elif "to" in bracket:
            # Split ranges like "1,501 to 2,000"
            parts = bracket.replace(",", "").split(" to ")
            return int(parts[0]), int(parts[1])
        else:
            try:
                # Handle cases where the value is a single number (e.g., "60")
                value = int(bracket.replace(",", ""))
                return value, value
            except ValueError:
                raise ValueError(f"Unexpected format in income bracket: {bracket}")

    # Apply parsing
    ehg_data[["min_income", "max_income"]] = ehg_data[income_column].apply(parse_income_bracket).apply(pd.Series)

    return ehg_data

# Example usage
ehg_couples = load_ehg_data("data/ehg_couples_families.csv", is_couple=True)
ehg_singles = load_ehg_data("data/ehg_singles.csv", is_couple=False)


In [28]:
is_couple = "Single"
annual_income = 50000
cpf_oa = 0
personal_cash = 0

In [29]:
total_income = annual_income + (spouse_income if is_couple == "Couple" else 0)
monthly_income = total_income / 12
total_cpf = cpf_oa + (spouse_cpf_oa if is_couple == "Couple" else 0)
total_cash = personal_cash + (spouse_cash if is_couple == "Couple" else 0)

# Load the appropriate EHG data
ehg_file = "data/ehg_couples_families.csv" if is_couple == "Couple" else "data/ehg_singles.csv"
ehg_data = load_ehg_data(ehg_file, is_couple=(is_couple == "Couple"))

# Find EHG amount based on income range
applicable_row = ehg_data[(ehg_data["min_income"] <= monthly_income) & (monthly_income <= ehg_data["max_income"])]
ehg_amount = applicable_row["EHG Amount (SGD)"].iloc[0] if not applicable_row.empty else 0

In [30]:
ehg_amount

5000

In [24]:
ehg_couples

,Average Monthly Household Income,EHG Amount (SGD),min_income,max_income
0,0 to 1500,120000,0,1500
1,1501 to 2000,110000,1501,2000
2,2001 to 2500,105000,2001,2500
3,2501 to 3000,95000,2501,3000
4,3001 to 3500,90000,3001,3500
5,3501 to 4000,80000,3501,4000
6,4001 to 4500,70000,4001,4500
7,4501 to 5000,65000,4501,5000
8,5001 to 5500,55000,5001,5500
9,5501 to 6000,50000,5501,6000


In [25]:
ehg_singles

,Half of Average Monthly Household Income,EHG (Singles) Amount (SGD),min_income,max_income
0,0 to 750,60000,0,750
1,751 to 1000,55000,751,1000
2,1001 to 1250,52500,1001,1250
3,1251 to 1500,47500,1251,1500
4,1501 to 1750,45000,1501,1750
5,1751 to 2000,40000,1751,2000
6,2001 to 2250,35000,2001,2250
7,2251 to 2500,32500,2251,2500
8,2501 to 2750,27500,2501,2750
9,2751 to 3000,25000,2751,3000


In [42]:
def calculate_hdb_loan_sg(monthly_income=14000, loan_tenure_years=25):
    """
    Calculate the maximum HDB loan amount under Singapore MSR rules:
    - MSR = 30% of gross monthly income
    - Interest Rate = 2.6% per annum
    """
    # MSR limit: 30% of monthly income
    msr_limit = 0.30 * monthly_income
    
    # Annual interest rate = 2.6%
    interest_rate = 0.026
    
    # Convert annual interest rate to monthly
    monthly_interest_rate = interest_rate / 12
    
    # Total number of monthly payments over the tenure
    num_payments = loan_tenure_years * 12
    
    # In case the interest rate is 0 or negative (edge case), use simple multiplication
    if monthly_interest_rate <= 0:
        max_loan = msr_limit * num_payments
    else:
        # Standard formula for Present Value of an annuity
        max_loan = msr_limit * (
            (1 - (1 + monthly_interest_rate) ** -num_payments) / monthly_interest_rate
        )
    
    return max_loan


def calculate_bank_loan_sg(monthly_income=14000, loan_tenure_years=25):
    """
    Calculate the maximum bank loan amount under Singapore TDSR rules:
    - TDSR = 55% of gross monthly income
    - Interest Rate = 3.0% per annum
    """
    # TDSR limit: 55% of monthly income
    tdsr_limit = 0.55 * monthly_income
    
    # Annual interest rate = 3.0%
    interest_rate = 0.03
    
    # Convert annual interest rate to monthly
    monthly_interest_rate = interest_rate / 12
    
    # Total number of monthly payments
    num_payments = loan_tenure_years * 12
    
    # Handle edge case if monthly interest rate is 0 or negative
    if monthly_interest_rate <= 0:
        max_loan = tdsr_limit * num_payments
    else:
        max_loan = tdsr_limit * (
            (1 - (1 + monthly_interest_rate) ** -num_payments) / monthly_interest_rate
        )
    
    return max_loan


In [43]:
monthly_income, loan_tenure_years, interest_rate = 14000, 25, 0.026

print(calculate_hdb_loan(monthly_income, loan_tenure_years, interest_rate))

monthly_income, loan_tenure_years, interest_rate = 14000, 25, 0.03
print(calculate_bank_loan(monthly_income, loan_tenure_years, interest_rate))

925784.0638118023
1623748.6907178522
